In [1]:
import boto3
import json
import re

### Mistral 7B

Prompt template for Mistral: https://www.promptingguide.ai/models/mistral-7b

In [2]:
SYSTEM_PROMPT = """<s>[INST] You are an assistant helping persons to book medical reservations. You should answer in the same language as the person who 
needs help. First, the person will tell you what he needs. Then, you will be given options for the reservation. 
The options will be presented as a list: [Option A, Option B, ...]. Notice that the options are in French because the customers are in France.
You will have to rank the best 3 options from all the listed options, with respect to the needs of the person, or all of them if there are less than 3.
If not explitally asked, you must not justify your answer.
The format for the answer, that you must imperatively follow, including the '[' and ']', is: [best option, 2nd best option, 3rd best option]\n"""

In [3]:
user_message = "I have some headeache, I want to visit a doctor."

In [4]:
TASK_PROMPT = f"""Here is the message of the user:\n\n{user_message}\n\nWhich type of specialist should this person see?
Here is the list of options: 
[Chirurgien dermatologique, 
Laser, 
Médecin généraliste, 
Spécialiste en chirurgie plastique reconstructrice et esthétique]
Do not explain your answer.
Do not forget to answer in the following format, without any addition: [best option, 2nd best option, 3rd best option][/INST]
["""

In [5]:
message = ''.join([SYSTEM_PROMPT, TASK_PROMPT])
prompt = json.dumps(message)
print(message)

<s>[INST] You are an assistant helping persons to book medical reservations. You should answer in the same language as the person who 
needs help. First, the person will tell you what he needs. Then, you will be given options for the reservation. 
The options will be presented as a list: [Option A, Option B, ...]. Notice that the options are in French because the customers are in France.
You will have to rank the best 3 options from all the listed options, with respect to the needs of the person, or all of them if there are less than 3.
If not explitally asked, you must not justify your answer.
The format for the answer, that you must imperatively follow, including the '[' and ']', is: [best option, 2nd best option, 3rd best option]
Here is the message of the user:

I have some headeache, I want to visit a doctor.

Which type of specialist should this person see?
Here is the list of options: 
[Chirurgien dermatologique, 
Laser, 
Médecin généraliste, 
Spécialiste en chirurgie plastique 

In [6]:
bedrock = boto3.client(service_name="bedrock-runtime")

body = json.dumps({
    "prompt": prompt,
    "max_tokens": 512,
    "top_p": 0.8,
    "temperature": 0.5,
})

modelId = "mistral.mistral-7b-instruct-v0:2"

accept = "application/json"
contentType = "application/json"

response = bedrock.invoke_model(
    body=body,
    modelId=modelId,
    accept=accept,
    contentType=contentType
)

# Read and parse the response body
response_body = json.loads(response.get('body').read())
response_text = response_body["outputs"][0]["text"]

stop = "]"
# Check if the stop token is present in the generated text
if stop in response_text:
    # If the stop token is found, truncate or modify the response
    truncated_text = response_text.split(stop)[0]  # Cut off at the stop token
    print(truncated_text)
else:
    # If the stop token isn't found, print the entire generated text
    print(response_text)

Médecin généraliste", "Spécialiste en médecine interne", "Chirurgien dermatologique"


It invents "Spécialiste en médecine interne"...

### Claude 3.5 Haiku

In [63]:
SYSTEM_PROMPT = """You are an assistant helping persons to book medical reservations. You should answer in the same language as the person who 
needs help. First, the person will tell you what he needs. Then, you will be given options for the reservation. 
The options will be presented as a list: [0. Option 0, 1. Option 1, ...]. Notice that the options are in French because the customers are in France.
You will have to rank the best 3 options from all the listed options, with respect to the needs of the person, or all of them if there are less than 3.
If not explitally asked, you must not justify your answer.
The format for the answer, that you must imperatively follow, including the '[' and ']', is: [index of best option, index of 2nd best option, index of 3rd best option].
The index begins at 0.\n"""

In [64]:
user_message = "I have some headeache, I want to visit a doctor."

In [65]:
options = ["Chirurgien dermatologique", "Laser", "Médecin généraliste", "Spécialiste en chirurgie plastique reconstructrice et esthétique"]
options = str([f"{i}. {option}" for i, option in enumerate(options)])
options = re.sub(r"'", "", options)
options = re.sub(r",", ",\n", options)
options

'[0. Chirurgien dermatologique,\n 1. Laser,\n 2. Médecin généraliste,\n 3. Spécialiste en chirurgie plastique reconstructrice et esthétique]'

In [66]:
TASK_PROMPT = f"""Here is the message of the user:\n\n{user_message}\n\nWhich option is the best?
Here is the list of options:
{options}
Do not explain your answer.
Do not forget to answer in the following format, without any addition: [index of best option, index of 2nd best option, index of 3rd best option]"""

In [67]:
message = ''.join([SYSTEM_PROMPT, TASK_PROMPT])
print(message)

You are an assistant helping persons to book medical reservations. You should answer in the same language as the person who 
needs help. First, the person will tell you what he needs. Then, you will be given options for the reservation. 
The options will be presented as a list: [0. Option 0, 1. Option 1, ...]. Notice that the options are in French because the customers are in France.
You will have to rank the best 3 options from all the listed options, with respect to the needs of the person, or all of them if there are less than 3.
If not explitally asked, you must not justify your answer.
The format for the answer, that you must imperatively follow, including the '[' and ']', is: [index of best option, index of 2nd best option, index of 3rd best option].
The index begins at 0.
Here is the message of the user:

I have some headeache, I want to visit a doctor.

Which option is the best?
Here is the list of options:
[0. Chirurgien dermatologique,
 1. Laser,
 2. Médecin généraliste,
 3. 

In [68]:
bedrock = boto3.client(service_name="bedrock-runtime")
body = json.dumps({
  "max_tokens": 256,
  "messages": [{"role": "user", "content": message}],
  "anthropic_version": "bedrock-2023-05-31"
})

response = bedrock.invoke_model(body=body, modelId="anthropic.claude-3-5-haiku-20241022-v1:0")

response_body = json.loads(response.get("body").read())
response_text = response_body['content'][0]['text']
print(response_text)

[2, 0, 1]


### LLM decision (Claude)

In [98]:
bedrock = boto3.client(service_name="bedrock-runtime")

In [99]:
def string_to_list(input_string):
    # Extract the content between brackets
    pattern = r'\[(.*)\]'
    match = re.search(pattern, input_string)
    
    if match:
        # Get the matched content
        content = match.group(1)
        
        # Split by comma and strip whitespace
        elements = [int(elem.strip()) for elem in content.split(',')]
        
        return elements
    else:
        return []

In [100]:
full_conversation = []

In [101]:
SYSTEM_PROMPT = """You are an assistant helping persons to book medical reservations. You should answer in the same language as the person who 
needs help. First, the person will tell you what he needs. Then, you will be given options for the reservation. 
The options will be presented as a list: [0. Option 0, 1. Option 1, ...]. Notice that the options are in French because the customers are in France.
You will have to rank the best 3 options from all the listed options, with respect to the needs of the person, or all of them if there are less than 3.
If not explicitly asked, you must not explain your answer.
The format for the answer, that you must imperatively follow, including the '[' and ']', is: [index of best option, index of the next best option, index of the next best option].
The index begins at 0.\n"""

In [102]:
user_message = "I have some headache, I want to visit a doctor." # retrieved from WhatsApp
CUSTOMER_PROMPT = f"""Here is the message of the user:\n\n{user_message}\n"""

In [103]:
full_conversation.append({"role": "user", "content": SYSTEM_PROMPT+CUSTOMER_PROMPT})
full_conversation

[{'role': 'user',
  'content': "You are an assistant helping persons to book medical reservations. You should answer in the same language as the person who \nneeds help. First, the person will tell you what he needs. Then, you will be given options for the reservation. \nThe options will be presented as a list: [0. Option 0, 1. Option 1, ...]. Notice that the options are in French because the customers are in France.\nYou will have to rank the best 3 options from all the listed options, with respect to the needs of the person, or all of them if there are less than 3.\nIf not explicitly asked, you must not explain your answer.\nThe format for the answer, that you must imperatively follow, including the '[' and ']', is: [index of best option, index of the next best option, index of the next best option].\nThe index begins at 0.\nHere is the message of the user:\n\nI have some headache, I want to visit a doctor.\n"}]

In [104]:
def llm_decision(options, current_conversation):
    """
    options: list of the available options
    current_conversation: context to allow the LLM to answer
    Notice that we could add a 'step' parameter to have a more relevant prompt.

    Returns the top 3 most likely answers and the updated conversation
    """
    # Formatting
    options = str([f"{i}. {option}" for i, option in enumerate(options)])
    options = re.sub(r"'", "", options)
    options = re.sub(r",", ",\n", options)

    # Prompt
    message = f"""Which option is the best?
Here is the list of options:
{options}
Do not explain your answer.
Do not forget to answer in the following format, without any addition: [index of best option, index of 2nd best option, index of 3rd best option]\n"""
    prompt = json.dumps(message)
    current_conversation.append({"role": "user", "content": message})
    body = json.dumps({
      "max_tokens": 256,
      "messages": current_conversation,
      "anthropic_version": "bedrock-2023-05-31"
    })

    # Generate answer
    response = bedrock.invoke_model(body=body, modelId="anthropic.claude-3-5-haiku-20241022-v1:0")
    response_body = json.loads(response.get("body").read())
    response_text = response_body['content'][0]['text']
    full_conversation.append({"role": "assistant", "content": response_text+'\n'})
    
    return string_to_list(response_text), full_conversation

In [105]:
options = ["Chirurgien dermatologique", "Laser", "Médecin généraliste", "Spécialiste en chirurgie plastique reconstructrice et esthétique"]

In [106]:
llm_decision(options, full_conversation)

([2, 0, 1],
 [{'role': 'user',
   'content': "You are an assistant helping persons to book medical reservations. You should answer in the same language as the person who \nneeds help. First, the person will tell you what he needs. Then, you will be given options for the reservation. \nThe options will be presented as a list: [0. Option 0, 1. Option 1, ...]. Notice that the options are in French because the customers are in France.\nYou will have to rank the best 3 options from all the listed options, with respect to the needs of the person, or all of them if there are less than 3.\nIf not explicitly asked, you must not explain your answer.\nThe format for the answer, that you must imperatively follow, including the '[' and ']', is: [index of best option, index of the next best option, index of the next best option].\nThe index begins at 0.\nHere is the message of the user:\n\nI have some headache, I want to visit a doctor.\n"},
  {'role': 'user',
   'content': 'Which option is the best

### Loop (Claude)